In [1]:
import os
from os.path import expanduser
import socket
import sys


# BELOW IS SYSTEM/PATH CONFIGURATION #
my_home = expanduser("~")
running_system = socket.gethostname()
# Configure machine's names here #
systemHLRN = 'blogin1'
systemLOCAL = 'Laurents-MacBook-Pro.local'
my_home = expanduser("~")
# Define local pass #
if running_system == systemLOCAL:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"Documents","projects", "awi-models")
elif running_system[0:1] == "b" or running_system == systemHLRN:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"awi-models")
else:
    print 'please configure your local machine : type socket.gethostname()'
print "working directory set to", wd_path
os.chdir(wd_path)
sys.path.append("codes/modules")
#########################################

computer =  bfn1001
working directory set to /home/hbkoziel/awi-models


In [2]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from pylab import *
from load_mesh_data_new import *
import numpy as np
import datetime as dt
from PiecewiseNorm import PiecewiseNorm
from netCDF4 import Dataset
import colormaps as cmaps
from matplotlib.colors import ListedColormap

In [3]:
# PLOT CONFIGURATION #

# load colormaps #
cmap = plt.get_cmap('RdBu_r')
# figure export 
export_plot = True
# figure export definition
dpicnt=150
# choose simulation
runid='Arc12'
# choose date
first_year = 1980
last_year  = 2015
years      = np.arange(first_year,last_year+1,1)
months =np.linspace(0,11,12).astype(int)
# choose depth
get3d = True
# load the given biological tracer #
var_id1, var_id2 = 'tr06', 'tr15'

########################

# AUTOMATIC DEFINITION OF PATHS
resultpath = '/scratch/usr/hbkvsk12/hlrn3_work2/results/'+runid+'/Oldfiles/'
savepath    = wd_path+'/figures/'+runid+'/'
meshpath    = wd_path+'/data/mesh/meshArc4.5/'

# Create figure directory if it does not exist
if os.path.exists(savepath) == False:
    try:
        os.mkdir(savepath)
        print ("Creation of the directory %s successfull" % savepath)
    except OSError:
        print ("Creation of the directory %s failed" % savepath)
else:
    print ("directory %s already existing" % savepath)
    
ncfile      = resultpath+runid+'.initial.mesh.diag.nc'
f           = Dataset(ncfile, 'r')
NodalVol3D = f.variables['cluster_vol'][:]

directory /home/hbkoziel/awi-models/figures/Arc12/ already existing


In [4]:
mesh = fesom_mesh(meshpath, get3d = get3d)
#mesh = load_mesh(meshpath)

Load 2d part of the grid in 3 second(s)
Load 3d part of the grid in 32 seconds


In [ ]:
# initlialyze
Chl2D   = np.zeros((len(years),len(mesh.x2)))
DATE = np.empty((len(years)))

# loop over years
for ind_year in range(0,len(years)):
     print years[ind_year]
     ncfile = resultpath+runid+'.'+str(years[ind_year])+'.oce.mean.nc'
     f      = Dataset(ncfile, 'r')
     chl1    = f.variables[var_id1][:,:].sum(axis=0)
     chl2    = f.variables[var_id2][:,:].sum(axis=0)
     if years[ind_year]<2000:
         Chl3D   = 30*(chl1+chl2)
     else:
         Chl3D   = 2*(chl1+chl2)
     Chl3D = Chl3D / 365
     #NPPtotal       = 365 * NPPtotal # Conversion from [mg/m2/day]   => [mg/m2/yr]
     #NPPtotal1 = np.sum(NPPtotal*NodalArea2D[:,0:mesh.n2d])/1e18 # [Pg C/year]
     for i, val in enumerate(Chl3D[0:len(mesh.x2)]):
        if i % 100000 == 0:
            print i
        ind      = mesh.n32[i,0:20]-1 # 20 = until 500m depth
        ind2     = ind[np.where(ind>-1)] # Only positive indices
        Chl2D[ind_year,i] = np.sum(Chl3D[ind2]*NodalVol3D[ind2])/np.sum(NodalVol3D[ind2])

1980
0
100000
200000
300000
400000
500000
600000
700000
1981
0
100000
200000
300000
400000
500000
600000
700000
1982
0
100000
200000
300000
400000
500000
600000
700000
1983
0
100000
200000
300000
400000
500000
600000
700000
1984
0
100000
200000
300000
400000
500000
600000
700000
1985
0
100000
200000
300000
400000
500000
600000
700000
1986
0
100000
200000
300000
400000
500000
600000
700000
1987
0
100000
200000
300000
400000
500000
600000
700000
1988
0
100000
200000
300000
400000
500000
600000
700000
1989
0
100000
200000
300000
400000
500000
600000
700000
1990
0
100000
200000
300000
400000
500000
600000
700000
1991
0
100000
200000
300000
400000
500000
600000
700000
1992
0
100000
200000
300000
400000
500000
600000
700000
1993
0
100000
200000
300000
400000
500000
600000
700000
1994
0
100000
200000
300000
400000
500000
600000
700000
1995
0
100000
200000
300000
400000
500000
600000
700000
1996
0
100000
200000
300000
400000
500000
600000
700000
1997
0
100000
200000
300000
400000
500000
600000

In [ ]:
print Chl2D.shape
do_export_csv = False
if do_export_csv == True:
    np.savetxt("/home/hbkoziel/awi-models/data/Arc12/annualintegratedChla.csv", Chl2D, delimiter=";")


In [ ]:
result_fit = np.ma.polyfit(years,Chl2D,1)

In [ ]:
elem2=mesh.elem#[mesh.no_cyclic_elem,:]
data2 = result_fit[0,:]
d=data2[elem2].mean(axis=1)

In [ ]:
contours = [-0.001, 0.001, 0.0001]
contours = np.arange(contours[0], contours[1]+contours[2], contours[2])
#contours=np.array([-600, -500, -400, -300, -200, -100, -80, -50, -30, -20, -10, -8, -5, -3, -2, -1, 0, 1, 2, 3, 5, 8, 10, 20, 30, 50, 80, 100, 200, 300, 400, 500, 600])
fig = plt.figure(figsize=(8,8), facecolor='w', edgecolor='k')
k = [i for (i, val) in enumerate(d) if not np.isnan(val)]
elem2=elem2[k,:]
print 'ftriplot, number of dummy points:', len(d)-len(k)		
map = Basemap(projection='nplaea',boundinglat=65,lon_0=0,resolution='l')
x, y = map(mesh.x2, mesh.y2)
plabels=[False,False,False,False]
mlabels=[True,True,True,True]    
map.drawparallels(np.arange(-80.,81.,20.), labels=plabels)
map.drawmeridians(np.arange(-180.,181.,20.),labels=mlabels) #[0,1,0,0]
map.drawmapboundary(fill_color='0.9')
map.fillcontinents(color='.5',lake_color='.7')
eps=(contours.max()-contours.min())/100.
data2[data2<=contours.min()]=contours.min()+eps
data2[data2>=contours.max()]=contours.max()-eps
im=plt.tricontourf(x, y, elem2, data2, levels=contours, cmap=cmap, norm=PiecewiseNorm(contours))
label = runid+': Annual integrated [Chla] trend ('+str(years[0])+' to '+str(years[len(years)-1])+')'
plt.title(label,y=1.05)
cbar=map.colorbar(im,"bottom", size="5%", pad="5%")
cbar.set_label(r'Annual Chla trend (mg(Chla) m$^{-2}$ year$^{-1}$)')
    
if export_plot == True:
    plt.savefig(savepath+'CHLintspatialArc_trend.png', dpi = dpicnt, bbox_inches='tight') 
        
plt.show() 